# Group Proposal - Predicting Following Day Rainfall in Australia


## Introduction

#### Background

Meteorological data has been collected and used to predict weather conditions ever since we have had the means to do so - since 1869! It helps one plan their day ahead and deal with the given weather conditions accordingly.

#### Central Question

Will it rain tomorrow for a given region in Australia based on a set of meteorological characteristics?

#### Dataset

The dataset that we will use is the “Rain in Australia” dataset by Joe Young and Adam Young. This dataset contains meteorological data across 10 years in Australia from 2007/10/31 to 2017/6/24 in various regions, collected by weather stations across Australia. The dataset contains variables such as weather conditions - wind speed, wind direction and temperature, as well the amount of precipitation in the form of rain on any given day.

## Preliminiary Exploratory Data Analysis

To aid in our decision for predictor variables, we can visualize which columns are present with the most valid data (least NA columns). A larger sample of data would allow us to reduce the impact of factors such as random error in the observation process and improve the overall quality of the analysis. We will select the three major cities we want to analyze and remove the categorical  variables from our dataset.

In [7]:
#load tidyverse
library(tidyverse)

In [8]:
#load data into r
weather_data_raw <- read_csv("weatherAUS.csv")

Rows: 145460 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Location, WindGustDir, WindDir9am, WindDir3pm, RainToday, RainTom...
dbl  (16): MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed,...
date  (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
# Select three cities for analysis

options(repr.plot.width =14, repr.plot.height = 8) 
count <- weather_data_raw |>
            filter(Location=="Sydney" | Location=="Melbourne" | Location=="Canberra") |>
            select(-WindGustDir, -WindDir9am, -WindDir3pm, -RainToday, -RainTomorrow, -Date, -Location) 

# Summarizes each row by counting non N/A cells. Renames the variables afterwards after conversion to data frame
output <- as.data.frame((colSums(!is.na(count)))) 
output <- cbind(rownames(output), output)
rownames(output) <- NULL
colnames(output) <- c("measurement","count")
output <- arrange(output, desc(count))
output

measurement,count
<chr>,<dbl>
WindSpeed3pm,9725
WindSpeed9am,9716
MaxTemp,9487
MinTemp,9483
Temp3pm,9478
Temp9am,9469
Humidity3pm,9461
Humidity9am,9413
Pressure3pm,9250


## Methods

## Expected Outcomes and Significance:

#### Expected Outcomes

#### Significance of Investigation

The significance of this analysis lies in the immense impact that weather and in particular, rain, has on society. Being able to predict rain is not only beneficial for day-to-day life but quintessential for industries such as agriculture, tourism, and urban development

#### Extended/Further Questions

Investigating precipitation further in the future can prompt inquiries on how rain patterns have evolved throughout the last decade. Furthermore, this investigation can also prompt further analysis into how accurate our model can be when compared to modern circumstances.